In [43]:
import numpy as np
import scipy.special
class neuralNetwork:
    def __init__(self , inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        self.activation_function = lambda x:scipy.special.expit(x)
        pass
    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        pass
    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
input_nodes = 28*28
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)
training_data_file = open("mldata/mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()
epochs = 5
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
test_data_file = open("mldata/mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    print(correct_label, "correct label")
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = np.argmax(outputs)
    print(label, "network's answer")
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

7 correct label
7 network's answer
2 correct label
2 network's answer
1 correct label
1 network's answer
0 correct label
0 network's answer
4 correct label
4 network's answer
1 correct label
1 network's answer
4 correct label
9 network's answer
9 correct label
4 network's answer
5 correct label
1 network's answer
9 correct label
9 network's answer
performance =  0.7


In [6]:
import numpy as np
import operator as op
def createDataSet():
    group = np.array([[1.0,1.1], [1.0,1.0], [0,0], [0,0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group, labels
def classify0(inX, dataset, labels, k):
    dataSetSize = dataset.shape[0]
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataset
    sqdiffMat = diffMat ** 2
    sqDistance = sqdiffMat.sum(axis=1)
    distance = sqDistance ** 0.5
    sortedDistIndicies = distance.argsort()
    classCount = {}
    for i in range(k):
        voteIable = labels[sortedDistIndicies[i]]
        classCount[voteIable] = classCount.get(voteIable, 0) + 1
    sortedClassCount = sorted(classCount.items(), key=op.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
group, labels = createDataSet()
classify0([0,0], group, labels, 3)
def file2matrix(filename):
    fr = open(filename)
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)
    returnMat = np.zeros((numberOfLines,3))
    classLabelVector = []
    index = 0
    for line in arrayOLines:
        line = line.strip()
        listFromLine = line.split('\t')
        returnMat[index,:] = listFromLine[0:3]
        classLabelVector.append(int(listFromLine[-1]))
        index += 1
    return returnMat, classLabelVector
datingDataMat, datingLabels = file2matrix('mldata/datingTestSet2.txt')
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet/np.tile(ranges, (m, 1))
    return normDataSet, ranges, minVals
normMat, ranges, minVals = autoNorm(datingDataMat)
def datingClassTest():
    hoRatio = 0.10
    datingDataMat, datingLabels = file2matrix('mldata/datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:], normMat[numTestVecs:m,:], datingLabels[numTestVecs:m], 3)
        #print(f"the classifier came back with: {classifierResult}, the real answer is {datingLabels[i]}")
        if classifierResult != datingLabels[i]:
            errorCount += 1.0
    print(f"the total error rate is {errorCount/float(numTestVecs)}")
datingClassTest()
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input("percentage of time spend playing video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per years?"))
    datingDataMat, datingLabels = file2matrix('mldata/datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = np.array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr - minVals)/ranges, normMat, datingLabels, 3)
    print(f"You will probably like this person: {resultList[classifierResult - 1]}")
classifyPerson()
def handwritingClassTest():
    hwLabels = []
    training_data_file = open("mldata/mnist_train_100.csv", 'r')
    training_data_list = training_data_file.readlines()
    training_data_file.close()
    m = len(training_data_list)
    trainingMat = np.zeros((m, 28 * 28))
    hwLabels = []
    for i in range(m):
        all_values = training_data_list[i].split(',')
        hwLabels.append(int(all_values[0]))
        trainingMat[i,:] = all_values[1:]
    test_data_file = open("mldata/mnist_test_10.csv", 'r')
    test_data_list = test_data_file.readlines()
    test_data_file.close()
    scorecard = []
    for record in test_data_list:
        all_values = record.split(',')
        correct_label = int(all_values[0])
        print(correct_label, "correct label")
        inputs = np.asfarray(all_values[1:])
        label = classify0(inputs, trainingMat, hwLabels, 3)
        print(label, "k-near's answer")
        if (label == correct_label):
            scorecard.append(1)
        else:
            scorecard.append(0)
    scorecard_array = np.asarray(scorecard)
    print ("performance = ", scorecard_array.sum() / scorecard_array.size)
handwritingClassTest()

the total error rate is 0.05
percentage of time spend playing video games?50
frequent flier miles earned per year?1000
liters of ice cream consumed per years?5
You will probably like this person: in large doses
7 correct label
7 k-near's answer
2 correct label
2 k-near's answer
1 correct label
1 k-near's answer
0 correct label
0 k-near's answer
4 correct label
4 k-near's answer
1 correct label
1 k-near's answer
4 correct label
9 k-near's answer
9 correct label
4 k-near's answer
5 correct label
6 k-near's answer
9 correct label
9 k-near's answer
performance =  0.7


In [30]:
import math
import operator
import pickle
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        labelCounts[currentLabel] = labelCounts[currentLabel] + 1 if currentLabel in labelCounts.keys() else 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * math.log(prob, 2)
    return shannonEnt
def createDataSet():
    dataSet = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet,labels
myDat,labels = createDataSet()
print(calcShannonEnt(myDat))
#myDat[0][-1]='maybe'
#print(calcShannonEnt(myDat))
def splitDataSet(dataSet, axis, value):
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet
print(splitDataSet(myDat,0,1))
print(splitDataSet(myDat,0,0))
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature
chooseBestFeatureToSplit(myDat)
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sort(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree
myTree = createTree(myDat, labels)
myDat,labels = createDataSet()
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:
                classLabel = secondDict[key]
    return classLabel
print(classify(myTree, labels, [1,0]))
print(classify(myTree, labels, [1,1]))
def storeTree(inputTree, filename):
    fw = open(filename, 'wb')
    pickle.dump(inputTree, fw, 2)
    fw.close()
def grabTree(filename):
    fr = open(filename, 'rb')
    return pickle.load(fr)
storeTree(myTree, 'mldata/tree.bin')
print(grabTree('mldata/tree.bin'))
fr = open('mldata/lenses.txt')
lenses = [inst.strip().split('\t') for inst in fr.readlines()]
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
lensesTree = createTree(lenses, lensesLabels)
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
print(classify(lensesTree, lensesLabels, ['young', 'hyper', 'yes', 'normal']))

0.9709505944546686
[[1, 'yes'], [1, 'yes'], [0, 'no']]
[[1, 'no'], [1, 'no']]
no
yes
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
hard


In [87]:
import numpy as np
import math
import re
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                  ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                  ['my', 'dalmation', 'is', 'so', 'cut', 'I', 'love', 'him'],
                  ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                  ['mr','licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                  ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]
    return postingList,classVec
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)] = 1
        else:
            print(f"the word: {word} is not in my Vocabulary！")
    return returnVec
def bagOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)] += 1
    return returnVec
listOPosts,listClasses=loadDataSet()
myVocabList = createVocabList(listOPosts)
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect,p1Vect,pAbusive
trainMat=[]
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
p0V,p1V,pAb=trainNB0(trainMat, listClasses)
print(myVocabList)
print(p1V)
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + math.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + math.log(1.0 - pClass1)
    return 1 if p1 > p0 else 0
def testingNB():
    listOPosts,listClasses=loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb=trainNB0(trainMat, listClasses)
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(f'{testEntry} classified as: {classifyNB(thisDoc, p0V, p1V, pAb)}')
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(f'{testEntry} classified as: {classifyNB(thisDoc, p0V, p1V, pAb)}')
testingNB()
def textParse(bigString):
    listOfTokens = re.split('\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]
def spamTest():
    docList=[]
    classList=[]
    fullText=[]
    for i in range(1,26):
        try:
            wordList = textParse(open(f'mldata/email/spam/{i}.txt', encoding='utf-8').read())
            docList.append(wordList)
            fullText.append(wordList)
            classList.append(1)
        except Exception as e:
            print(e)
        try:
            wordList = textParse(open(f'mldata/email/ham/{i}.txt', encoding='utf-8').read())
            docList.append(wordList)
            fullText.append(wordList)
            classList.append(0)
        except Exception as e:
            print(e)
    vcabList = createVocabList(docList)
    trainingSet = list(range(45))
    testSet = []
    for i in range(10):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat=[]
    trainClasses=[]
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vcabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    with open('mldata/bayes.bin', 'wb') as f:
        pickle.dump(p0V, f)
        pickle.dump(p1V, f)
        pickle.dump(pSpam, f)
    with open('mldata/bayes.bin', 'rb') as f:
        p0V = pickle.load(f)
        p1V = pickle.load(f)
        pSpam = pickle.load(f)
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vcabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print(f'the error rate is: {float(errorCount)/len(testSet)}')
spamTest()

['is', 'quit', 'I', 'stop', 'ate', 'garbage', 'to', 'so', 'licks', 'has', 'take', 'worthless', 'love', 'mr', 'steak', 'dalmation', 'please', 'cut', 'maybe', 'posting', 'buying', 'dog', 'him', 'problems', 'help', 'park', 'food', 'stupid', 'my', 'not', 'flea', 'how']
[-3.04452244 -2.35137526 -3.04452244 -2.35137526 -3.04452244 -2.35137526
 -2.35137526 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -1.94591015
 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -3.04452244
 -2.35137526 -2.35137526 -2.35137526 -1.94591015 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -2.35137526 -1.65822808 -3.04452244 -2.35137526
 -3.04452244 -3.04452244]
['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1
'utf-8' codec can't decode byte 0x92 in position 884: invalid start byte
'utf-8' codec can't decode byte 0x92 in position 66: invalid start byte
'utf-8' codec can't decode byte 0xae in position 199: invalid start byte
the error rate is: 0.0


In [40]:
import numpy as np
import pickle
def loadDataSet():
    dataMat = []
    labelMat = []
    fr = open('mldata/testSet.txt')
    for line in fr.readlines():
        lineArr = line.strip().split()
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(int(lineArr[2]))
    return dataMat,labelMat
def sigmoid(inX):
    return 1.0/(1+np.exp(-inX))
def gradAscent(dataMatIn,classLabels):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    m,n = np.shape(dataMatrix)
    alpha = 0.001
    maxCycles = 500
    weights = np.ones((n,1))
    for k in range(maxCycles):
        h = sigmoid(dataMatrix*weights)
        error = labelMat - h
        weights = weights + alpha * dataMatrix.transpose() * error
    return weights
dataArr,labelMat = loadDataSet()
print(gradAscent(dataArr,labelMat))
def stocGradAscent0(dataMatrix, classLabels, numIter=150):
    m,n = np.shape(dataMatrix)
    alpha = 0.01
    weights = np.ones(n)
    for j in range(numIter):
        for i in range(m):
            h = sigmoid(sum(dataMatrix[i] * weights))
            error = classLabels[i] - h
            weights = weights + alpha * error * dataMatrix[i]
    return weights
print(stocGradAscent0(np.array(dataArr),labelMat))
def stocGradAscent1(dataMatrix, classLabels, numIter=150):
    m,n = np.shape(dataMatrix)
    weights = np.ones(n)
    for j in range(numIter):
        dataIndex = list(range(m))
        for i in range(m):
            alpha = 4/(1.0+j+i) + 0.01
            randIndex = int(np.random.uniform(0,len(dataIndex)))
            h = sigmoid(sum(dataMatrix[randIndex] * weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
            del(dataIndex[randIndex])
    return weights
print(stocGradAscent1(np.array(dataArr),labelMat))
def classifyVector(inX, weights):
    prob = sigmoid(sum(inX*weights))
    if prob > 0.5:
        return 1.0
    else:
        return 0.0
def colicTest():
    frTrain=open('mldata/horseColicTraining.txt')
    frTest=open('mldata/horseColicTest.txt')
    trainingSet=[]
    trainingLabels=[]
    for line in frTrain.readlines():
        currLine = line.strip().split('\t')
        lineArr=[]
        for i in range(21):
            lineArr.append(float(currLine[i]))
        trainingSet.append(lineArr)
        trainingLabels.append(float(currLine[21]))
    trainWeights = None
    try:
        with open('mldata/logistic.bin', 'rb') as f:
            trainWeights = pickle.load(f)
    except:
        print("can't find logistic.bin")
    if type(trainWeights) != np.ndarray:
        trainWeights = stocGradAscent1(np.array(trainingSet),trainingLabels,500)
        with open('mldata/logistic.bin', 'wb') as f:
            pickle.dump(trainWeights,f)
    errorCount=0
    numTestVec=0.0
    for line in frTest.readlines():
        numTestVec += 1.0
        currLine = line.strip().split('\t')
        lineArr=[]
        for i in range(21):
            lineArr.append(float(currLine[i]))
        if int(classifyVector(np.array(lineArr),trainWeights)) != int(currLine[21]):
            errorCount += 1
    errorRate=float(errorCount)/numTestVec
    print(f'the error rate of this test is: {errorRate}')
    return errorRate
def multiTest():
    numTests=10
    errorSum=0.0
    for k in range(numTests):
        errorSum += colicTest()
    print(f'after {numTests} iterations the average error rate is: {errorSum/float(numTests)}')
multiTest()

[[ 0.08108752]
 [-0.1233496 ]]
[ 0.08344415 -0.17862418]
[-0.05977823 -0.23760366]
can't find logistic.bin


C:\Users\dangz\AppData\Local\Temp\ipykernel_9232\3973054505.py:13: RuntimeWarning: overflow encountered in exp
  return 1.0/(1+np.exp(-inX))


the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
the error rate of this test is: 0.29850746268656714
after 10 iterations the average error rate is: 0.29850746268656714


In [19]:
import numpy as np
def loadDataSet(filename):
    numFeat = len(open(filename).readline().split('\t')) - 1
    dataMat = []
    labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split('\t')
        for i in range(numFeat):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return dataMat,labelMat
def standRegres(xArr, yArr):
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    xTx = xMat.T*xMat
    if np.linalg.det(xTx) == 0.0:
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I * (xMat.T*yMat)
    return ws
xArr,yArr = loadDataSet('mldata/ex0.txt')
ws = standRegres(xArr,yArr)
yHat = np.mat(xArr) * ws
print(np.corrcoef(yHat.T, np.mat(yArr)))
def lwlr(testPoint,xArr,yArr,k=1.0):
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    m = np.shape(xMat)[0]
    weights = np.mat(np.eye(m))
    for j in range(m):
        diffMat = testPoint - xMat[j,:]
        weights[j,j] = np.exp(diffMat * diffMat.T/(-2.0*k**2))
    xTx = xMat.T * (weights * xMat)
    if np.linalg.det(xTx) == 0.0:
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I * (xMat.T*(weights*yMat))
    return testPoint * ws
def lwlrTest(testArr, xArr, yArr, k = 1.0):
    m = np.shape(testArr)[0]
    yHat = np.zeros(m)
    for i in range(m):
        yHat[i] = lwlr(testArr[i], xArr, yArr, k)
    return yHat
print(np.corrcoef(lwlrTest(xArr, xArr, yArr, 0.003).T, np.mat(yArr)))
def rssError(yArr, yHatArr):
    return ((yArr - yHatArr)**2).sum()
abX,abY=loadDataSet('mldata/abalone.txt')
print(rssError(abY[0:99],lwlrTest(abX[0:99],abX[0:99],abY[0:99],0.1)))
print(rssError(abY[0:99],lwlrTest(abX[0:99],abX[0:99],abY[0:99],1)))
print(rssError(abY[0:99],lwlrTest(abX[0:99],abX[0:99],abY[0:99],10)))
print(rssError(abY[100:199],lwlrTest(abX[100:199],abX[0:99],abY[0:99],0.1)))
print(rssError(abY[100:199],lwlrTest(abX[100:199],abX[0:99],abY[0:99],1)))
print(rssError(abY[100:199],lwlrTest(abX[100:199],abX[0:99],abY[0:99],10)))
def ridgeRegres(xMat, yMat, lam=0.2):
    xTx = xMat.T*xMat
    denom = xTx + np.eye(np.shape(xMat)[1]) * lam
    if np.linalg.det(denom) == 0.0:
        print("This matrix is singular, cannot do inverse")
        return
    ws = denom.I * (xMat.T * yMat)
    return ws
def ridgeTest(xArr, yArr):
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    yMean = np.mean(yMat, 0)
    yMat = yMat - yMean
    xMeans = np.mean(xMat, 0)
    xVar = np.var(xMat, 0)
    xMat = (xMat - xMeans)/xVar
    numTestPts = 1
    wMat = np.zeros((numTestPts, np.shape(xMat)[1]))
    for i in range(numTestPts):
        ws = ridgeRegres(xMat, yMat, np.exp(i-10))
        wMat[i,:] = ws.T
    return wMat
print(ridgeTest(abX, abY))
def stageWise(xArr, yArr, eps=0.01, numIt=100):
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    yMean = np.mean(yMat, 0)
    yMat = yMat - yMean
    xMeans = np.mean(xMat, 0)
    xVar = np.var(xMat, 0)
    xMat = (xMat - xMeans)/xVar
    m,n=np.shape(xMat)
    returnMat = np.zeros((numIt,n))
    ws = np.zeros((n,1))
    wsTest = ws.copy()
    wsMax = ws.copy()
    for i in range(numIt):
        print(ws.T)
        lowestError = np.inf
        for j in range(n):
            for sign in [-1,1]:
                wsTest = ws.copy()
                wsTest[j] += eps * sign
                yTest = xMat * wsTest
                rssE = rssError(yMat.A, yTest.A)
                if rssE < lowestError:
                    lowestError = rssE
                    wsMax = wsTest
        ws = wsMax.copy()
        returnMat[i,:]=ws.T
    return returnMat
print(stageWise(abX, abY, 0.01, 200))

[[1.         0.98647356]
 [0.98647356 1.        ]]
[[1.         0.99931945]
 [0.99931945 1.        ]]
56.8118936813369
429.89056187020685
549.1181708824906
94927.34165777494
573.5261441896996
517.5711905384079
[[ 0.04304419 -0.02274163  0.13214088  0.02075182  2.22403745 -0.99895298
  -0.11725424  0.16622922]]
[[0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.   0.   0.   0.01 0.   0.   0.   0.  ]]
[[0.   0.   0.   0.02 0.   0.   0.   0.  ]]
[[0.   0.   0.   0.03 0.   0.   0.   0.  ]]
[[0.   0.   0.   0.04 0.   0.   0.   0.  ]]
[[0.   0.   0.   0.05 0.   0.   0.   0.  ]]
[[0.   0.   0.   0.06 0.   0.   0.   0.  ]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.  ]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.01]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.02]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.03]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.04]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.05]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.06]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.07]]
[[0.   0.   0.01 0.06 0.   0.   0.   0.08]]


In [110]:
import numpy as np
import math
def loadDataSet(filename):
    dataMat = []
    fr = open(filename)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine))
        dataMat.append(fltLine)
    return dataMat
def distEclud(vecA, vecB):
    #print("dist", math.sqrt(np.sum(np.power(vecA - vecB, 2))))
    return math.sqrt(np.sum(np.power(vecA - vecB, 2)))
def randCent(dataSet, k):
    n = np.shape(dataSet)[1]
    centroids = np.mat(np.zeros((k,n)))
    for j in range(n):
        minJ = np.min(dataSet[:,j])
        rangeJ = float(np.max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * np.random.rand(k,1)
    return centroids
datMat = np.mat(loadDataSet('mldata/unsupervised/testSet.txt'))
print(np.min(datMat[:,0]))
print(np.min(datMat[:,1]))
print(np.max(datMat[:,0]))
print(np.max(datMat[:,1]))
print(randCent(datMat, 2))
print(distEclud(datMat[0], datMat[1]))
def kMeans(dataSet, k, distMeas = distEclud, createCent = randCent):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m, 2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = np.inf
            minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist
        #print(centroids)
        for cent in range(k):
            pstInClust = dataSet[np.nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = np.mean(pstInClust, axis=0)
    return centroids, clusterAssment
cent, cluster = kMeans(datMat, 4)
def biKmeans(dataSet, k, distMeas = distEclud):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataSet, axis=0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(np.mat(centroid0), dataSet[j,:])
    while len(centList) < k:
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[np.nonzero(clusterAssment[:,0].A==i)[0],:]
            centroidMat, splitClusterAss = kMeans(ptsInCurrCluster, 2, distMeas)
            print(splitClusterAss.shape)
            sseSplit = np.sum(splitClusterAss[:,1])
            sseNotSplit = np.sum(clusterAssment[np.nonzero(clusterAssment[:,0].A!=i)[0], 1])
            print(f'sseSplit: {sseSplit}, sseNotSplit: {sseNotSplit}')
            if sseSplit + sseNotSplit < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClusterAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[np.nonzero(bestClustAss[:,0].A==1)[0], 0] = len(centList)
        bestClustAss[np.nonzero(bestClustAss[:,0].A==0)[0], 0] = bestCentToSplit
        print(f'bestCentToSplit: {bestCentToSplit}, len(bestClustAss): {len(bestClustAss)}')
        centList[bestCentToSplit] = bestNewCents[0,:].A.flatten().tolist()
        centList.append(bestNewCents[1,:].A.flatten().tolist())
        print(centList)
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return np.mat(centList), clusterAssment
datMat3 = np.mat(loadDataSet('mldata/unsupervised/testSet2.txt'))
biKmeans(datMat3, 3)

-5.379713
-4.232586
4.838138
5.1904
[[-0.4420751   1.94059639]
 [-0.16448688 -0.01128562]]
5.184632816681332
(60, 2)
sseSplit: 144.82021724250035, sseNotSplit: 0.0
bestCentToSplit: 0, len(bestClustAss): 60
[[-0.006756050000000002, 3.2271029749999998], [-0.45965614999999993, -2.7782156000000002]]
(40, 2)
sseSplit: 49.61872046270045, sseNotSplit: 21.798183738423
(20, 2)
sseSplit: 14.698143155806289, sseNotSplit: 123.02203350407736
bestCentToSplit: 0, len(bestClustAss): 40
[[-2.94737575, 3.3263781000000003], [-0.45965614999999993, -2.7782156000000002], [2.93386365, 3.12782785]]


(matrix([[-2.94737575,  3.3263781 ],
         [-0.45965615, -2.7782156 ],
         [ 2.93386365,  3.12782785]]),
 matrix([[2.        , 0.38139356],
         [0.        , 0.82475077],
         [1.        , 1.0108639 ],
         [2.        , 1.15995155],
         [0.        , 1.16351392],
         [1.        , 1.96765728],
         [2.        , 0.9150191 ],
         [0.        , 0.46916551],
         [1.        , 1.88098128],
         [2.        , 2.72778511],
         [0.        , 2.29797746],
         [1.        , 0.16021061],
         [2.        , 1.05804787],
         [0.        , 0.4097449 ],
         [1.        , 1.45510908],
         [2.        , 1.22325471],
         [0.        , 2.22177461],
         [1.        , 0.09883066],
         [2.        , 0.36394209],
         [0.        , 0.79959117],
         [1.        , 0.97045965],
         [2.        , 1.31318515],
         [0.        , 0.86641953],
         [1.        , 0.38572737],
         [2.        , 1.73329036],
         [0.